<a href="https://githubtocolab.com/gee-community/geemap/blob/master/docs/notebooks/135_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Earth Engine Image Segmentation with the Segment Anything Model**

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [1]:
!pip install -U geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00


In [2]:
%pip install segment-geospatial pycrs

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cub

In [1]:
import ee
import geemap
from samgeo import SamGeo

In [10]:
# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='my-project')

True

# New Section

In [11]:
# Create a map centered on New Zealand
Map = geemap.Map()
# Define the area of interest in EPSG:2193 coordinates
roi = ee.Geometry.Rectangle([5729127, 1991311, 5732957, 1997472],'EPSG:2193',geodesic=False)

# Add ROI to the map (converted to WGS84 for visualization)
Map.addLayer(roi.transform('EPSG:4326'), {'color': 'red'}, 'ROI')
Map.centerObject(roi.transform('EPSG:4326'), 14)

# Get Sentinel-1 GRD collection
collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(roi)
    .filterDate('2020-01-01', '2023-01-01')
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('resolution_meters', 10))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
)

# Select the first image with VV polarization
image = collection.select('VV').first()

# Add SAR image to the map with stretching
vis_params = {
    'min': -20,
    'max': 0,
    'palette': ['black', 'white']
}
Map.addLayer(image, vis_params, 'Sentinel-1 VV')

EEException: Caller does not have required permission to use project my-project. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=my-project and then retry. Propagation of the new permission may take a few minutes.

In [ ]:
# Convert ROI coordinates to list for export
bbox = roi.bounds().getInfo()['coordinates'][0]
bbox = [coord[0] for coord in bbox[:2]] + [coord[1] for coord in bbox[2:]]

In [ ]:
geemap.ee_to_geotiff(
    image, "naip.tif", bbox, zoom=17, vis_params={"bands": ["R", "G", "B"]}
)

In [ ]:
# Export the image as GeoTIFF
geemap.ee_to_geotiff(
    image=image,
    filename="s1.tif",
    region=roi,
    scale=10,
    crs='EPSG:2193',
    file_per_band=False
)

In [ ]:
sam.generate("naip.tif", output="masks.tif", foreground=True, unique=True)

In [ ]:
sam.show_masks(cmap="binary_r")

In [ ]:
sam.show_anns(axis="off", alpha=1, output="annotations.tif")

In [ ]:
Map.add_raster("annotations.tif", opacity=0.5, layer_name="Masks")
Map

In [ ]:
sam.tiff_to_vector("masks.tif", "masks.shp")

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
Map.add_vector("masks.shp", layer_name="Vector", style=style)
Map